jupyter notebook --NotebookApp.iopub_data_rate_limit=10000000

In [ ]:
import networkx as nx
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
import random
import copy
import heapq

In [ ]:
df = pd.read_csv('C:/Projects IT/Python/Projet Metaheuristic/project.csv', sep =';')
nameTable = pd.read_csv('C:/Projects IT/Python/Projet Metaheuristic/indexStation.csv', sep =';')

df.head(3)

In [ ]:
nameTable.head(3)

In [ ]:
MG = nx.MultiGraph()

In [ ]:
for i in range(0, len(df.Station)):
    MG.add_edge(df.Station[i], df.Connection[i], weight=df.Time[i])

In [ ]:
MG

In [ ]:
print(MG.adj)

In [ ]:
nx.draw(MG, label=True)
plt.show()

In [ ]:
shortestMulti = nx.johnson(MG)

In [ ]:
class Chromosome:
    def __init__(self, nodes):
        self.nodes = nodes       
    
    def shuffleGenes(self):
        random.shuffle(self.nodes)   
    
    #Order Changing Mutation, 1 gene is selected thanks to Pm probability then swapped with the other node(cant duplicate)
    def mutationGene(self, Pm):
        self.Pm = Pm
        for i in range(0, len(self.nodes)):
            self.randMutate = random.random()
            if self.randMutate < self.Pm:
                self.mutateTo = random.randint(0,len(self.nodes)-1)
                self.tmpGene = copy.copy(self.nodes[i])
                self.nodes = np.where(self.nodes == self.mutateTo, self.tmpGene, self.nodes)
                self.nodes[i] = self.mutateTo

    def pathWay(self, Multigraph): #Multigraph come from the Johnson Networkx calculation for shortest path between 2 nodes
        self.Multigraph = Multigraph
        self.totalPath = []
        for i in range(0, len(self.nodes)-1):        
            if i+1 is not None:
                self.totalPath = merge(self.totalPath, self.Multigraph[self.nodes[i]][self.nodes[i+1]])   
        return self.totalPath
        
    def __repr__(self):
        self.strNodes = "Chromosome :"
        for i in self.nodes:
            self.node = str(i)
            self.strNodes = self.strNodes + "|" + self.node
        self.strNodes = self.strNodes + "|"
        return self.strNodes

In [ ]:
#Crossover PErmutation Encoding (1)ABC DEF + (2)DAB CFE --> ABC(1) DFE (in the order of the 2d chromosome genes)
def crossoverPermutation(chromo1, chromo2):
    lenChromo = int(len(chromo1.nodes) / 2)
    childrenChr = chromo1.nodes[0:lenChromo]
    for i in chromo2.nodes:
        if i not in childrenChr:
            childrenChr = np.append(childrenChr, [i])
    return childrenChr
    

In [ ]:
def populationGeneration(popsize, nodeList):
    populationOpti = []
    for i in range(0,popsize,1):
        nodeList = np.unique(nodeList, axis=0)
        random.shuffle(nodeList)
        populationOpti = populationOpti + [Chromosome(nodeList)]
    return populationOpti

In [ ]:
#function can merge 2 list where last element of one(a) is the first element of the second(b)
#for example [1,2,3,4] + [4,5,6] = [1,2,3,4,5,6]
def merge(a, b):
    c = a[:-1] + b[:]
    return c
    

In [ ]:
def fitnessScore(uniqueNodeList, chromosome, MultiG):
    fitScore = len(uniqueNodeList.unique()) / len(chromosome.pathWay(MultiG))
    return fitScore

In [ ]:
def nameFit(uniqueNodeName, chromosome, MultiG):
    NameList = np.array([])
    for i in chromosome.pathWay(MultiG):
        NameList = np.append(NameList, uniqueNodeName[i])
    NameFreq = np.unique(NameList, return_counts=True)
    NameFreq[1]
    score = 1 / np.max(NameFreq[1])
    return score

In [ ]:
def nodesFit(chromosome, MultiG):
    score = 1 / len(chromosome.pathWay(MultiG))
    return score
    

In [ ]:
#MAIN LOOP
population = populationGeneration(1000, df.Station)
for globalLoob in range(0,5):
    for z in range(0,10):
        fitList = np.array([])
        for i in range(0,len(population)):
            fitList = np.append(fitList, nodesFit(population[i], shortestMulti))
        IndexfitBest = heapq.nlargest(500, range(len(fitList)), fitList.take)
        population_best = np.take(population, IndexfitBest)
        child = np.array([Chromosome(crossoverPermutation(population_best[0], population_best[1]))])
        for u in range(1, len(population_best)):
            child = np.append(child, Chromosome(crossoverPermutation(population_best[u-1], population_best[u])))
        population = np.append(child, population_best)
    for uu in range(0,len(population)-1):
        population[uu].mutationGene(0.01)
    print(fitList)
    print(population[0])

print(fitList)
print(population)
print(population_best)
print(fitList)

        
        
        
    
  

In [ ]:
len(population[5].pathWay(shortestMulti))

In [ ]:
population